# Import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, make_scorer, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, brier_score_loss, log_loss, mean_absolute_error, median_absolute_error
from sklearn.metrics import auc as sklearn_auc
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
print(df.columns)

# Check for NaN values in each column
missing_values = df.isna().sum()

# Filter columns that have at least one missing value
columns_with_missing_values = missing_values[missing_values > 0]

print("Columns with missing values and their count:")
print(columns_with_missing_values)



# Data preparation

In [ ]:
# Load your dataset
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()



# # Assuming 'no WD and no LPA' represents untreated customers
#df = df[df["welcome_discount"] == 1]


# # Check if the DataFrame is not empty
# if not untreated_df.empty:
#     # Define your features and target variable
#     X = untreated_df.drop(['policy_nr_hashed', 'last_data_year', 'churn', 'control_group', 'premiums', 'last_brand', 'last_type', 'last_fuel_type', 'last_postcode', 'last_product', 'last_trend_nr_coverages', 'last_change_premium_abs', 'last_change_premium_perc', 'years_since_last_car_change'], axis=1)
#     y = untreated_df['churn']

#     # Split the dataset into training and test sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# else:
#     print("No untreated customers found. Check the filtering criteria.")


In [ ]:
ategorical_features = []
continuous_features = []
binary_features = []

# Define a threshold for the maximum number of unique values for a categorical column
max_unique_values_for_categorical = 10

# List the columns with fewer NaNs
#columns_with_fewer_nans = ['last_split', 'last_vs_first_split', 'cum_change_premium_perc']

# Drop rows with NaNs in these specific columns
#df = df.dropna(subset=columns_with_fewer_nans)

# Iterate through each column to determine if it's categorical, continuous, or binary
for column in df.columns:
    unique_values = df[column].nunique()
    if unique_values == 2:
        # If exactly 2 unique values, treat column as binary
        binary_features.append(column)
    elif (df[column].dtype == 'object' or unique_values <= max_unique_values_for_categorical) and unique_values > 2:
        # If object type or up to the threshold of unique values (and more than 2), treat as categorical
        categorical_features.append(column)
    else:
        # Otherwise, treat as continuous
        continuous_features.append(column)

print(f'Binary Features: {binary_features}')
print(f'Categorical Features: {categorical_features}')
print(f'Continuous Features: {continuous_features}')

for cat in categorical_features:
     df[cat] = df[cat].astype("category")

In [ ]:
#df = pd.get_dummies(df, columns=categorical_features)
# List of columns to exclude from the features
excluded_columns = ['churn', 'welcome_discount', 'last_trend_nr_coverages', 'last_change_premium_abs', 'last_change_premium_perc', 'years_since_last_car_change' ]

# Assuming 'binary_features' and 'continuous_features' are lists of your binary and continuous columns
selected_features = binary_features + continuous_features

# Select only the binary and continuous features, excluding the specified columns
X = df[[col for col in df.columns if col in selected_features and col not in excluded_columns]]
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


# Setting up GridsearchCV

In [ ]:

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    # Add more parameters here if needed
}

# test

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV or BayesSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# bayes_search = BayesSearchCV(rf, search_spaces, n_iter=32, cv=5, n_jobs=-1) # Uncomment for BayesSearchCV



# Setting up Hyperopt

In [ ]:

# Define the search space
space = {
    'n_estimators': hp.choice('n_estimators', range(100, 301)),
    'max_depth': hp.choice('max_depth', [None, 5, 10, 15]),
    'min_samples_split': hp.choice('min_samples_split', range(2, 11)),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 6)),
    'max_leaf_nodes': hp.choice('max_leaf_nodes', [None, 10, 20, 30, 40, 50]),
    # 'class_weight': 'balanced' is set directly in the model initialization
}


# Define the objective function
def objective(params):
    # Include class_weight='balanced' in the model
    rf = RandomForestClassifier(**params, class_weight='balanced', random_state=42)
    best_score = cross_val_score(rf, X_train, y_train, scoring='neg_brier_score', cv=5).mean()
    return {'loss': -best_score, 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=32, trials=trials, rstate=np.random.seed(42))

# Convert index values to actual values for the hyperparameters
best_params['n_estimators'] = range(100, 301)[best_params['n_estimators']]
best_params['max_depth'] = [None, 5, 10, 15, 20][best_params['max_depth']]
best_params['min_samples_split'] = range(2, 11)[best_params['min_samples_split']]
best_params['min_samples_leaf'] = range(1, 6)[best_params['min_samples_leaf']]
best_params['max_leaf_nodes'] = [None, 10, 20, 30, 40, 50][best_params['max_leaf_nodes']]

print("Best parameters:", best_params)



# Training the model

In [ ]:
# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


In [ ]:
# Assuming 'best_params' contains the best parameters found by Hyperopt
# Create the RandomForestClassifier with the best parameters
rf_best = RandomForestClassifier(**best_params, random_state=42)

# Fit the model to the training data
rf_best.fit(X_train, y_train)

# Evaluate the model on the training set (optional, for comparison)
train_score = rf_best.score(X_train, y_train)
print("Training set score:", train_score)

# Evaluate the model on the test set
test_score = rf_best.score(X_test, y_test)
print("Test set score:", test_score)


# Evaluate the model

In [ ]:
# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)  # Ensure to use the transformed version of X_test if applicable

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
# Calculate metrics with 'weighted' average for multiclass classification
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (Weighted): {precision}")
print(f"Recall (Weighted): {recall}")
print(f"F1 Score (Weighted): {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

In [ ]:
# Use the best parameters from Hyperopt and add class_weight='balanced'
rf_best = RandomForestClassifier(
    n_estimators=286,
    max_depth=None,
    max_leaf_nodes=None,
    min_samples_leaf=2,
    min_samples_split=10,
    class_weight='balanced',  # Adjust class weights
    random_state=42
)

# Fit the model to the training data
rf_best.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_best.predict(X_test)

# # Calculate metrics
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1 = f1_score(y_test, y_pred, average='weighted')
# conf_matrix = confusion_matrix(y_test, y_pred)

# # Print the metrics
# print(f"Accuracy: {accuracy}")
# print(f"Precision (Weighted): {precision}")
# print(f"Recall (Weighted): {recall}")
# print(f"F1 Score (Weighted): {f1}")
# print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate metrics without 'weighted' average (using 'macro' average as an example)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)


# Calculate the AUC-PR
y_scores = rf_best.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
precision, recall, _ = precision_recall_curve(y_test, y_scores)
auc_pr = sklearn_auc(recall, precision)

# Calculate negative brier
neg_brier_score = -brier_score_loss(y_test, y_scores)

# Print the metrics
print(f'ROC AUC Score: {auc}')
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"AUC-PR: {auc_pr}")
print(f"Negative Brier Score: {neg_brier_score}")

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import brier_score_loss, log_loss, make_scorer
import numpy as np

# Define custom scoring functions
def brier_score_loss_func(y_true, y_pred_probs):
    return brier_score_loss(y_true, y_pred_probs[:, 1])

def log_loss_func(y_true, y_pred_probs):
    return log_loss(y_true, y_pred_probs[:, 1])

# Convert custom scoring functions to scorers
brier_scorer = make_scorer(brier_score_loss_func, needs_proba=True)
log_loss_scorer = make_scorer(log_loss_func, needs_proba=True)

# Initialize cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store scores
brier_scores = []
log_loss_scores = []
mae_scores = []
medae_scores = []

# Perform cross-validation
for train_idx, test_idx in cv.split(X_train, y_train):
    # Split data using .iloc for Pandas DataFrame/Series
    X_cv_train, X_cv_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_cv_train, y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Fit model
    rf_best.fit(X_cv_train, y_cv_train)

    # Predict probabilities
    y_pred_probs = rf_best.predict_proba(X_cv_test)

    # Calculate scores
    brier_scores.append(brier_score_loss(y_cv_test, y_pred_probs[:, 1]))
    log_loss_scores.append(log_loss(y_cv_test, y_pred_probs))
    mae_scores.append(mean_absolute_error(y_cv_test, y_pred_probs[:, 1]))
    medae_scores.append(median_absolute_error(y_cv_test, y_pred_probs[:, 1]))

# Calculate average scores
avg_brier = np.mean(brier_scores)
avg_log_loss = np.mean(log_loss_scores)
avg_mae = np.mean(mae_scores)
avg_medae = np.mean(medae_scores)

# Print average scores
print(f"Average Brier Score: {avg_brier}")
print(f"Average Log Loss: {avg_log_loss}")
print(f"Average MAE: {avg_mae}")
print(f"Average MedAE: {avg_medae}")